# text snippets etc to database

In [1]:
# %env CUDA_VISIBLE_DEVICES = 4

In [2]:
import ffmpeg
import cv2
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from moviepy.editor import VideoFileClip
from nist_database import MSSQLDatabase
from video_tools import generate_audio, generate_gps, get_checksum
import os
import json
import whisper
import pandas as pd
from io import StringIO
from datetime import datetime
from pytz import timezone
import time
import logging

In [3]:
COMMIT_EVERYTHING = False   # whether to commit to db
DELETE_DUPS = True         # whether to exclude files with checksums already in db
FOLDER_TO_ADD = 'assets/data/video/'
MODEL_TYPE = "medium.en"    # tiny.en, base.en, small.en, medium.en, large

assert FOLDER_TO_ADD[-1:] == '/'
logging.basicConfig(format='', filename='nist.log', level=logging.INFO)

# Batching Video Files & Deleting Dup's

In [4]:
%%time

local_files = os.listdir(FOLDER_TO_ADD)
local_vid_files = [vid for vid in local_files if vid.split('.')[-1].lower() == 'mp4']
video_files = [FOLDER_TO_ADD + vid for vid in local_vid_files]
# video_files = [vid for vid in video_files if '093' in vid] # delete after

for vid in video_files:
    assert os.path.exists(vid), f'Video file {vid} does not exist'

audio_files = [generate_audio(vid_file, "wav") or vid_file[:-3] + 'wav' for vid_file in video_files]
gps_files = [generate_gps(vid_file, "go-pro") or vid_file[:-3] + 'csv' for vid_file in video_files]


CPU times: user 823 µs, sys: 1.94 ms, total: 2.76 ms
Wall time: 1.47 ms


In [5]:
ffmpeg_binaries = '/home/idies/workspace/nist_ai/extras/ffmpeg-6.0-amd64-static'
assert os.path.exists(ffmpeg_binaries)
os.environ["PATH"] = os.environ["PATH"] + f":{ffmpeg_binaries}"
with open('/home/idies/workspace/nist_ai/Henry/nist-ai.json','r') as f:
    AUTH = json.load(f)
DB=MSSQLDatabase(AUTH,'NIST_AI')

# delete duplicate videos by checksum
unique_video_files, unique_audio_files = [], []
for video_file, audio_file in zip(video_files, audio_files):
    checksum = get_checksum(video_file)
    df = DB.execute_query(f"select * from video where checksum='{checksum}'")
    if len(df.index) > 0:
        dup_names = []
        for i in df.index:
            dup_names.append(df.loc[0]['filename'])
        print(f"{video_file} has {'duplicates' if len(dup_names) > 1 else 'a duplicate'} called \n" \
              f"{dup_names[0] if len(dup_names) == 1 else ', '.join(dup_names)}\n")
        continue
    unique_video_files.append(video_file)
    unique_audio_files.append(audio_file)
if DELETE_DUPS: video_files, audio_files = unique_video_files, unique_audio_files
print(video_files)


assets/data/video/hernandez-perfect-game-2012.mp4 has a duplicate called 
assets/data/video/hernandez-perfect-game-2012.mp4

assets/data/video/usa-japan-championship-2023.mp4 has a duplicate called 
assets/data/video/usa-japan-championship-2023.mp4

assets/data/video/henderson-hr-june-11-2023.mp4 has a duplicate called 
assets/data/video/henderson-hr-june-11-2023.mp4

assets/data/video/VIDEO_7152.mp4 has a duplicate called 
assets/data/video/VIDEO_7152.mp4

assets/data/video/budavari_GX_0101.MP4 has a duplicate called 
assets/data/video/budavari_GX_0101.MP4

assets/data/video/budavari_GX_0120.MP4 has a duplicate called 
assets/data/video/budavari_GX_0120.MP4

assets/data/video/budavari_GX_0124.MP4 has a duplicate called 
assets/data/video/budavari_GX_0124.MP4

[]


# Querying Tables

### deleting video

### adding table row

### changing constraints

# Prepare database & Whisper Model

In [32]:
# check if tables already exist
sql_table = lambda x: f"""
SELECT * FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
WHERE TABLE_NAME='{x}'
"""

table_df = DB.execute_query(sql_table('video'))
if len(table_df.index) == 0:
    ddl = """
    create table video (
        id bigint identity(1,1) not null
      , filename varchar(1024) not null
      , checksum varchar(36)
      , metadata varchar(max) null    -- JSON encoded dict metadata
      , constraint pk_gemdmodel primary key(id)
    )"""
    DB.execute_update(ddl)

table_df = DB.execute_query(sql_table('audio'))
if len(table_df.index) == 0:
    ddl = """
        create table audio (
        id bigint identity(1,1) not null
      , video_id bigint
      , filename varchar(1024) not null
      , checksum varchar(36)
      , constraint pk_audio primary key(id)
      , constraint fk_audio_video foreign key (video_id) REFERENCES video(id) ON DELETE CASCADE
    )"""
    DB.execute_update(ddl)

table_df = DB.execute_query(sql_table('transcription'))
if len(table_df.index) == 0:
    ddl = """
      CREATE TABLE transcription (
      id bigint identity(1,1) not null
    , audio_id bigint not null
    , config varchar(max)          -- JSON string 
    , constraint pk_text_run primary key(id)
    , constraint fk_text_run_audio foreign key (audio_id) REFERENCES audio(id) ON DELETE CASCADE
    )"""
    DB.execute_update(ddl)

table_df = DB.execute_query(sql_table('text_segment'))
if len(table_df.index) == 0:
    ddl = """
      CREATE TABLE text_segment(
      id bigint identity(1,1) not null
    , transcription_id bigint not null
    , video_id bigint null
    , segment varchar(max)
    , temperature real
    , time_start real
    , time_end real
    , constraint pk_text_segment primary key(id)
    , constraint fk_text_segment_text_run foreign key (transcription_id) REFERENCES transcription(id) ON DELETE CASCADE
    )"""
    DB.execute_update(ddl)

table_df = DB.execute_query(sql_table('word_segment'))
if len(table_df.index) == 0:
    ddl = """
      CREATE TABLE word_segment(
      id bigint identity(1,1) not null
    , transcription_id bigint not null
    , word varchar(max)
    , probability real
    , time_start real
    , time_end real
    , constraint pk_word_segment primary key(id)
    , constraint fk_word_segment_text_run foreign key (transcription_id) REFERENCES transcription(id) ON DELETE CASCADE
    )"""
    DB.execute_update(ddl)

table_df = DB.execute_query(sql_table('gps'))
if len(table_df.index) == 0:
    ddl = """
      CREATE TABLE gps(
      id bigint identity(1,1) not null
    , video_id bigint not null
    , location varchar(max) not null
    , timestamp datetime
    , latitude float
    , longitude float
    , altitude float
    , constraint pk_gps primary key(id)
    , constraint fk_gps foreign key (video_id) REFERENCES video(id) ON DELETE CASCADE
    )"""
    DB.execute_update(ddl)

In [9]:
Base = automap_base()
Base.prepare(DB.ENGINE, reflect=True)
# for c in Base.classes:
#     print(c)

Video = Base.classes.video
Audio = Base.classes.audio
Transcription = Base.classes.transcription
TextSegment = Base.classes.text_segment
WordSegment = Base.classes.word_segment
GPSPing = Base.classes.gps
session = Session(DB.ENGINE)

In [10]:
!nvidia-smi

Sun Aug 27 19:25:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   32C    P0    54W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:62:00.0 Off |                    0 |
| N/A   

In [11]:
%%time
model = whisper.load_model(MODEL_TYPE, download_root='/home/idies/workspace/nist_ai/extras/whisper-models')

CPU times: user 36.6 s, sys: 12.3 s, total: 48.9 s
Wall time: 36.1 s


# Videos

In [12]:
# add each video file to db
videos = []
for video_file in video_files:
    metadata_dict = ffmpeg.probe(video_file)
    metadata = json.dumps(metadata_dict)
    checksum = get_checksum(video_file)
    videos.append(Video(checksum=checksum, filename=video_file, metadata=metadata))
    if COMMIT_EVERYTHING:
        session.add(videos[-1])
        session.commit()

# Audio

In [13]:
# add each audio file to db
audios = []
for audio_file, video in zip(audio_files, videos):
    audios.append(Audio(video=video, filename=audio_file, checksum=get_checksum(audio_file)))
    if COMMIT_EVERYTHING:
        session.add(audios[-1])
        session.commit()

# Transcription, TextSegment, WordSegment

In [14]:
%%time

# add each transcription to db
start_time = time.time()
results, transcriptions = [], []
for audio in audios:
    results.append(model.transcribe(audio.filename, word_timestamps=True))
    config_obj = {'model': 'whisper-net', 'load_model': MODEL_TYPE}
    transcriptions.append(Transcription(audio=audio, config=json.dumps(config_obj)))
    if COMMIT_EVERYTHING:
        session.add(transcriptions[-1])
        session.commit()
    logging.info(f"{datetime.now(timezone('EST')).strftime('%m/%d/%Y %H:%M:%S')}: transcribed \"{audio.filename.split('/')[-1]}\"")
    logging.info(f"... w/ {MODEL_TYPE} ({round(time.time() - start_time, 1)} secs)")


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.87 µs


In [15]:
%%time

# add text- and word- segments to db
for result, video, transcription, video_file in zip(results, videos, transcriptions, video_files):
    vidcap = cv2.VideoCapture(video_file)
    for segment in result["segments"]:
        
        # add word segments
        for word_dict in segment['words']:
            
            # make utf-8 solution
            pretty_word = ''.join([c.lower() for c in word_dict['word'].strip() if 97 <= ord(c.lower()) <= 97 + 25])
            if not pretty_word: continue
            start, end, prob = word_dict['start'], word_dict['end'], word_dict['probability']
            # print(f"|{word_dict['word']}| -> |{pretty_word}|")
            print(pretty_word, end=' ')
            word_row = WordSegment(transcription=transcription, word=pretty_word, probability=prob, \
                        time_start=start, time_end=end)
            if COMMIT_EVERYTHING: session.add(word_row)
            
        # get thumbnail binary
        photo_bytes = None
        milliseconds = int(segment['start'] * 1000)
        vidcap.set(cv2.CAP_PROP_POS_MSEC, milliseconds)
        success, image = vidcap.read()
        if success:
            success, buffer = cv2.imencode('.jpg', image)  # get image as binary
            photo_bytes = buffer.tobytes()  # convert to bytes
        
        # add text segment
        text_segment_row = TextSegment(transcription=transcription, video_id = video.id, thumbnail=photo_bytes, time_start=segment['start'], \
                    time_end=segment['end'], segment=segment['text'])
        if COMMIT_EVERYTHING: session.add(text_segment_row)
    if COMMIT_EVERYTHING: session.commit()
    print('\n\n', '-' * 3, '\n')

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.34 µs


# GPS Metadata

In [16]:
# add gps metadata to db
for gps_file, video, video_file in zip(gps_files, videos, video_files):
    if not gps_file: continue
    if not os.path.exists(gps_file): 
        if COMMIT_EVERYTHING: 
            pass
        logging.warning("... no csv file w/ gps points found")
        continue

    with open(gps_file, 'r') as f:
        # delete all lines that start with '#' or ' ', add to df
        lines = [line for line in f.readlines() if line[0] not in ['#', ' ']]
        df = pd.read_csv(StringIO(''.join(lines)))
        if COMMIT_EVERYTHING: 
            pass
        logging.info(f"... found {df.shape[0]} gps points in csv")

        # for every row in df, add json string of its elements
        gps_info = []
        for i in df.index:
            gps_info.append(json.dumps(df.loc[i].to_dict()))
        
        # get timestamps from pd series
        gps_timestamps_epoch = df['UTC Time'].values
        gps_timestamps = []
        for i, timestamp in enumerate(gps_timestamps_epoch):
            timestamp = datetime.fromtimestamp(timestamp) # defaults correctly to utc
            timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            gps_timestamps.append(timestamp)
        latitudes, longitudes, altitudes = df['Latitude'], df['Longitude'], df['Altitude (m)']

        # append items in df to gps table
        for info_dict, timestamp, lat, lon, alt in zip(gps_info, gps_timestamps, latitudes, longitudes, altitudes):
            gps_ping = GPSPing(video=video, location=info_dict, timestamp=timestamp, \
                                       latitude=lat, longitude=lon, altitude=alt)
            if COMMIT_EVERYTHING: session.add(gps_ping)
        if COMMIT_EVERYTHING: session.commit()